# <b> LIBRARIES

In [1]:
import pandas as pd
import numpy as np
!pip3 install yfinance
import yfinance as yf
import datetime as dt
!pip3 install pandas_datareader
from pandas_datareader import data as pdr
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import os
from pandas import ExcelWriter
import datetime as dt 

In [2]:
yf.pdr_override()

# <b> SCREENER

In [4]:
stocklist.head(2)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,#,Symbol,...,Avg AT Margin 3Q,Avg AT Margin 4Q,Avg AT Margin 5Q,Avg AT Margin 6Q,AT Margin Accel,EPS Due Date,Avg Sales % Chg 3Q,Shrt Int % of Float,Shrt Int % Chg,% Chg YTD
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,INMD,...,38.1,28.4,29.5,29.6,0,02-18-2020,52.8,23.0,3.4,24.74
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,ENPH,...,16.2,13.6,12.0,10.4,1,02-26-2020,83.5,35.2,10.6,25.14


In [3]:
filePath=r'RichardStocks.xlsx'
stocklist = pd.read_excel(filePath)
stock=str(stocklist["Symbol"][0])
stock

'INMD'

In [4]:
start =dt.datetime(2017,12,1)
now = dt.datetime.now()

filePath=r'RichardStocks.xlsx'


stocklist = pd.read_excel(filePath)

exportList= pd.DataFrame(columns=[
    'Stock', "RS_Rating", "50 Day MA", "150 Day Ma", 
    "200 Day MA", "52 Week Low", "52 week High"
])


for i in stocklist.index:
    stock=str(stocklist["Symbol"][i])
    RS_Rating=stocklist["RS Rating"][i]
    
    try:
        df = pdr.get_data_yahoo(stock, start, now)
        
        smaUsed=[50,150,200]
        for x in smaUsed:
            sma=x
            
        df["SMA_"+str(sma)]=round(df.iloc[:,4].rolling(window=sma).mean(),2)
                  
        currentClose=df["Adj Close"][-1]
        MA50=df["SMA_50"][-1]
        MA150=df["SMA_150"][-1]
        MA200=df["SMA_200"][-1]
        min_52week=min(df["adj Close"][-260:])
        max_52week=max(df["adj Close"][-260:])
        try:
            MA_200_20 = df['SMA_200'][-20]
        except Exception:
            MA_200_20=0

        print("Checking "+stock+".....")

        #Condition 1: Current Price > 150 SMA and > 200 SMA
        if(currentClose>MA_150>MA_200):
            cond_1=True
        else:
            cond_1=False
            
        #Condition 2: 150 SMA and > 200 SMA
        if(MA_150>MA_200):
            cond_2=True
        else:
            cond_2=False
        
        #Condition 3: 200 SMA trending up for at least 1 month (ideally 4-5 months)
        if(MA_200>MA_200_20):
            cond_3=True
        else:
            cond_3=False
        
        #Condition 4: 50 SMA> 150 SMA and 50 SMA> 200 SMA
        if(MA_50>MA_150>MA_200):
            cond_4=True
        else:
            cond_4=False
        
        #Condition 5: Current Price > 50 SMA
        if(currentClose>MA_50):
            cond_5=True
        else:
            cond_5=False
        
        #Condition 6: Current Price is at least 30% above 52 week low (Many of the best are up 100-300% before coming out of consolidation)
        if(currentClose>=(1.3*min_52week)):
            cond_6=True
        else:
            cond_6=False
        
        #Condition 7: Current Price is within 25% of 52 week high
        if(currentClose>=(.75*max_52week)):
            cond_7=True
        else:
            cond_7=False
        
        #Condition 8: IBD RS rating >70 and the higher the better
        if(RS_Rating>70):
            cond_8=True
        else:
            cond_8=False
            
        if(cond_1 and cond_2 and cond_3 and cond_4 and cond_5 and cond_6 and cond_7 and cond_8):
            exportList = exportList.append({
                "Stock": stock, 
                "RS_Rating": RS_Rating, 
                "50 Day MA": MA_50, 
                "150 Day Ma": MA_150, 
                "200 Day MA": MA_200, 
                "52 Week Low": min_52week, 
                "52 week High": max_52week}, 
                ignore_index=True)
            print(stock + ' is tradable')

    except Exception:
        print("No data on "+stock)

print(exportList)

newFile=os.path.dirname(filePath)+"/ScreenOutput.xlsx"

writer= ExcelWriter(newFile)
exportList.to_excel(writer,"Sheet1")
writer.save()

[*********************100%***********************]  1 of 1 completed
No data on INMD
[*********************100%***********************]  1 of 1 completed
No data on ENPH
[*********************100%***********************]  1 of 1 completed
No data on PLMR
[*********************100%***********************]  1 of 1 completed
No data on VCTR
[*********************100%***********************]  1 of 1 completed
No data on SEDG
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GSX: No data found, symbol may be delisted
No data on GSX
[*********************100%***********************]  1 of 1 completed
No data on ARES
[*********************100%***********************]  1 of 1 completed
No data on PFSI
[*********************100%***********************]  1 of 1 completed
No data on ACMR
[*********************100%***********************]  1 of 1 completed
No data on PAAS
[*********************100%***********************]  1 of 1 completed
No data on DXCM
[

KeyboardInterrupt: 